In [4]:
import numpy as np
from numpy import *
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


Using TensorFlow backend.


In [5]:
def split_data(X, y, test_data_size):
    return train_test_split(X, y, test_size=test_data_size, random_state=42)


def reshape_data(arr, img_rows, img_cols, channels):
    return arr.reshape(arr.shape[0], img_rows, img_cols, channels)

def cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus):
    model = Sequential()

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     strides=1,
                     input_shape=(img_rows, img_cols, channels)))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    nb_filters = 64
    kernel_size = (4, 4)

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    # model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    # model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    nb_filters = 128
    kernel_size = (8, 8)

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    # model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    # model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(12, 12)))

    model.add(Flatten())
    print("Model flattened out to: ", model.output_shape)

    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(4096))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    #model = multi_gpu_model(model, gpus=nb_gpus)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print(model.summary())

    stop = EarlyStopping(monitor='acc',
                         min_delta=0.001,
                         patience=2,
                         verbose=0,
                         mode='auto')

    #tensor_board = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

    model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
              verbose=1,
              validation_split=0.2,
              class_weight='auto',
              #callbacks=[stop, tensor_board]
              )
    return model

In [3]:
if __name__ == '__main__':

    batch_size = 100
    nb_classes = 189
    nb_epoch = 10
    nb_gpus = 8
    img_rows, img_cols = 512, 512
    channels = 1
    nb_filters = 32
    kernel_size = (2, 2)

    # Import data
    labels = pd.read_csv("../3kdata.csv")
    print(labels.values)
    
    X = np.load("X_sample.npy")
    #X = genfromtxt('../new_data1.csv', delimiter=',')

    y = labels.FindingLabels
    # y = np.array(pd.get_dummies(y))
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    y = y.reshape(-1, 1)

    a=y;
    print(a)
    print("Splitting data into test/ train datasets")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    print("Reshaping Data")
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)

    print("X_train Shape: ", X_train.shape)
    print("X_test Shape: ", X_test.shape)

    input_shape = (img_rows, img_cols, channels)

    print("Normalizing Data")
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train /= 255.0
    X_test /= 255.0

    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    print("y_train Shape: ", y_train.shape)
    print("y_test Shape: ", y_test.shape)


    model = cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus)
    
    print("Training Acc : ")
    score = model.evaluate(X_train, y_train, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    print("Testing Acc : ")
    score = model.evaluate(X_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
   

NameError: name 'pd' is not defined

In [19]:
print(a.max())

188


In [6]:
model.save("DiseasePredictorModel.tfl")

In [9]:
from keras.models import load_model
model=load_model('DiseasePredictorModel.tfl')
model.summary()
print("Testing Acc : ")
score = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 511, 511, 32)      160       
_________________________________________________________________
activation_1 (Activation)    (None, 511, 511, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 510, 510, 32)      4128      
_________________________________________________________________
activation_2 (Activation)    (None, 510, 510, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 509, 509, 32)      4128      
_________________________________________________________________
activation_3 (Activation)    (None, 509, 509, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 254, 32)     

In [20]:
print(labels)

         ImageIndex             FindingLabels  Follow-up  PatientID  \
0  00018387_035.png     Effusion|Infiltration         35      18387   
1  00018387_036.png              Infiltration         36      18387   
2  00018387_037.png                No Finding         37      18387   
3  00018388_000.png  Atelectasis|Infiltration          0      18388   
4  00018388_001.png                No Finding          1      18388   

   PatientAge PatientGender ViewPosition  OriginalImage[Width  Height]  \
0          59             M           PA                 2992     2991   
1          59             M           PA                 3056     2516   
2          59             M           PA                 2992     2991   
3          76             M           PA                 2992     2991   
4          77             M           PA                 2992     2991   

   OriginalImagePixelSpacing[x     y]  
0                        0.143  0.143  
1                        0.139  0.139  
2       

In [21]:
print(y)

[[1]
 [2]
 [3]
 [0]
 [3]]


In [6]:
import time

import cv2
import numpy as np
import pandas as pd


def convert_images_to_arrays(file_path, df):
    lst_imgs = [l for l in df['ImageIndex']]

    return np.array([np.array(cv2.resize(cv2.imread(file_path + img, cv2.IMREAD_GRAYSCALE),(512,512))) for img in lst_imgs])


def save_to_array(arr_name, arr_object):
    return np.save(arr_name, arr_object)


if __name__ == '__main__':
    start_time = time.time()

    labels = pd.read_csv("../3kdata.csv")

    print("Writing Train Array")
    X_train = convert_images_to_arrays('3kdata/', labels)

    #print(labels)
    #print(X_train.shape)

    print("Saving Train Array")
    save_to_array('X_sample.npy', X_train)

    print("Seconds: ", round(time.time() - start_time), 2)

Writing Train Array
Saving Train Array
Seconds:  68 2
